In [9]:
import pandas as pd
import numpy as np
import pickle as pk
import re
import warnings

from gensim.models import FastText
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine
from nltk import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import os
PATH = '/Users/yoonjeong/Desktop/Lab_project/'
os.chdir(PATH)

In [3]:
fasttext = FastText.load('3.Modeling/fasttext2.model')

In [4]:
with open("3.Modeling/fasttext2_embedding.pickle", 'rb') as fr:
    fasttext_embedding = pk.load(fr)

In [5]:
user_input = "甲은 乙로부터 토지를 매수하기로 계약을 체결한 후 계약금 및 중도금을 지급하고 나머지 잔금을 지급하지 않은 상태에서 乙에게 형질변경 및 건축허가를 받는 데에 반드시 필요하니 부동산매도용인감증명서 및 확인서면을 甲에게 건네주면 위 용도로만 사용하겠다고 거짓말하여, 이에 속은 乙로부터 즉석에서 부동산매도용인감증명서 및 등기의무자본인확인서면을 교부받은 후 이를 이용하여 甲명의로 위 토지의 소유권이전등기를 경료하였습니다. 이 경우 甲을 사기죄로 처벌할 수 있는지요?"

In [18]:
df = pd.read_csv("2.Preprocessing/fraud_prep.csv", encoding="cp949")

In [6]:
from hanjahangul import *
user_input_hanja = hanja_to_hangul_dueum(user_input)

In [11]:
clean_user_input = re.sub("[^가-힣. ]", '', user_input_hanja)
clean_user_input

'갑은 을로부터 토지를 매수하기로 계약을 체결한 후 계약금 및 중도금을 지급하고 나머지 잔금을 지급하지 않은 상태에서 을에게 형질변경 및 건축허가를 받는 데에 반드시 필요하니 부동산매도용인감증명서 및 확인서면을 갑에게 건네주면 위 용도로만 사용하겠다고 거짓말하여 이에 속은 을로부터 즉석에서 부동산매도용인감증명서 및 등기의무자본인확인서면을 교부받은 후 이를 이용하여 갑명의로 위 토지의 소유권이전등기를 경료하였습니다. 이 경우 갑을 사기죄로 처벌할 수 있는지요'

In [12]:
user_sentence_tokened = sent_tokenize(clean_user_input)
user_sentence_tokened

['갑은 을로부터 토지를 매수하기로 계약을 체결한 후 계약금 및 중도금을 지급하고 나머지 잔금을 지급하지 않은 상태에서 을에게 형질변경 및 건축허가를 받는 데에 반드시 필요하니 부동산매도용인감증명서 및 확인서면을 갑에게 건네주면 위 용도로만 사용하겠다고 거짓말하여 이에 속은 을로부터 즉석에서 부동산매도용인감증명서 및 등기의무자본인확인서면을 교부받은 후 이를 이용하여 갑명의로 위 토지의 소유권이전등기를 경료하였습니다.',
 '이 경우 갑을 사기죄로 처벌할 수 있는지요']

In [13]:
embedding = None
count = 0
for sentence in user_sentence_tokened:
    if embedding is None:
        embedding = fasttext.wv.get_sentence_vector(sentence)
    else:
        embedding += fasttext.wv.get_sentence_vector(sentence)
    count += 1

f_embedding = embedding / count

In [14]:
f_similarity = cosine_similarity([f_embedding], fasttext_embedding)
f_similarity = list(enumerate(f_similarity[0]))
f_similarity = sorted(f_similarity, key=lambda x : x[1], reverse = True)

In [19]:
precidents = []

for i in range(5):
    idx = f_similarity[i][0]
    sim = round(f_similarity[i][1] * 100, 2)
    
    precidents.append([df['판례일련번호'][idx], df['사건명'][idx], df['사건종류명'][idx], df['판례내용'][idx], sim])

In [20]:
f_result = pd.DataFrame(precidents, columns=['판례일련번호', '사건명', '사건종류명', '판례내용', '유사도'])
f_result

,판례일련번호,사건명,사건종류명,판례내용,유사도
0,104187,사기,형사,【피 고 인】 【상 고 인】 피고인【원심판결】 \n서울형사지방법원 1987.7....,95.23
1,86031,사기피고,형사,"【상 고 인】 검사【원심판결】 \n제1심 대구지방법원 안동지원, 제2심 대구고등...",95.22
2,76900,사기피고사건,형사,"【피 고 인】 【주 문】 피고인은 무죄【이 유】 이 사건 공소사실은,...",95.15
3,98769,사기,형사,【피 고 인】 【상 고 인】 피고인【원심판결】 \n서울형사지방법원 1983.10...,95.11
4,100360,사기,형사,【피 고 인】 【상 고 인】 검사【원심판결】 서울형사지방법원 1984.12.5....,95.07


In [25]:
f_result.판례내용[0]

'【피 고 인】 【상 고 인】   피고인【원심판결】 \n서울형사지방법원 1987.7.18 선고, 87노2462 판결【주    문】  원심판결을 파기하고 사건을 서울형사지방법원 합의부에 환송한다.【이    유】  피고인의 상고이유에 대하여,  원심이 유지한 제1심판결의 범죄사실에 의하면, 피고인은 주식회사 통일의 영업관리과 채권담당자인데 위 회사에서 피해자 윤금만에게 할부로 매각한 선반기계 1대에 대한 할부금을 피해자가 제대로 불입치 아니하여 동인을 상대로 기계인도소송을 제기, 승소판결을 받아 1986.7.3 그 인도집행까지 마쳤으나 피해자가 피고인에게 금 120만원을 보관시키면서 회사간부를 설득하여 위 금원만으로 기계를 타에 처분환가치 말고 반환하여 줄 것을 수차 요구하여옴을 기화로 그해 7.5 위 금원만으로 동인에게 기계를 반환하여 줄 의사나 능력이 없음에도 불구하고 위 금원을 회사에 입금 처리하여 기계할부잔대금 일부라도 일단 회수하고 보자는 생각에서 위 기계를 반환하여 주겠으니 금 120만원을 회사에 입금시키라고 말하여 이에 속은 피해자로 부터 즉석에서 동의를 얻어 그 시경 피고인이 받아 보관중이던 위의 금원을 위 회사에 입금, 위 회사로 하여금 위 금액을 교부받게 하여 이를 편취한 것이라는 것이다.  그리고 위의 범죄사실을 인정한 증거로서 피고인과 증인 윤금만의 공판정 진술, 피고인과 윤금만에 대한 검사의 피의자신문조서 및 진술조서를 들고있다.  위의 증거를 종합하여 볼 때 주식회사 통일이 피해자에게 1985.12.27 선반기계 1대를 금 5,572,600원에 판매함에 있어 당일 금 120만원을 계약금으로 받고 기계를 인도하고 나머지 대금에 대하여서는 1986.1월부터 6개월에 걸쳐 분할상환하기로 약정하였는데 피해자가 그 지급을 하지 아니하기 때문에 소송을 거쳐 그해 7.3 인도집행까지 마쳤던 바, 피해자가 위 회사의 채권관계 실무자인 피고인에게 금 120만원을 제공하면서 기계를 처분하지 말고 이를 피해자에게 돌려주어 사업을 운영하게끔 해달라는 간청을 한